In [19]:
from transformers import AutoModel, AutoTokenizer
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import zipfile
from transformers import AutoTokenizer, AutoModel
import torch

from sklearn.metrics.pairwise import cosine_similarity
import faiss

import string

In [2]:
# Path to the zip file
zip_file_path = r'C:\Users\John\Desktop\Patent Datasets\final_data\HiDrive\patentmatch_train_balanced.zip'
zip_file_path_test = r'C:\Users\John\Desktop\Patent Datasets\final_data\HiDrive\patentmatch_test_balanced.zip'

# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as z:
    # List all files in the zip
    print(z.namelist())
    
    # Assuming the zip contains a TSV file named 'data.tsv'
    with z.open('patentmatch_train_balanced.tsv') as f:
        df_train_cites = pd.read_csv(f, delimiter='\t')

# Open the zip file
with zipfile.ZipFile(zip_file_path_test, 'r') as z:
    # List all files in the zip
    print(z.namelist())
    
    # Assuming the zip contains a TSV file named 'data.tsv'
    with z.open('patentmatch_test_balanced.tsv') as f:
        df_test_cites = pd.read_csv(f, delimiter='\t')


['patentmatch_train_balanced.tsv']
['patentmatch_test_balanced.tsv']


In [3]:
# Ensure that only necessary columns are selected
df_train_ct = df_train_cites[["text", "text_b", "label"]]
df_test_ct = df_test_cites[["text", "text_b", "label"]]

In [4]:
# Assuming df_train_ct and df_test_ct are pandas DataFrames
# Convert them to Hugging Face Datasets
train_dataset = Dataset.from_pandas(df_train_ct)
test_dataset = Dataset.from_pandas(df_test_ct)

In [6]:
model_name = "princeton-nlp/sup-simcse-bert-base-uncased"  # or another SimCSE model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Binary classification


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at princeton-nlp/sup-simcse-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
def preprocess_function(examples):
    return tokenizer(
        examples["text"],
        examples["text_b"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

# Remove unnecessary columns for training
train_dataset = train_dataset.remove_columns(["text", "text_b"])
test_dataset = test_dataset.remove_columns(["text", "text_b"])

# Set the format for PyTorch
train_dataset.set_format("torch")
test_dataset.set_format("torch")


Map:   0%|          | 0/278304 [00:00<?, ? examples/s]

Map:   0%|          | 0/69576 [00:00<?, ? examples/s]

In [8]:

training_args = TrainingArguments(
    output_dir="./results",  # Directory for saving checkpoints
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,  # Only keep the last 2 checkpoints
    logging_dir="./logs",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)


c:\Users\John\miniconda3\envs\llama\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\John\AppData\Local\Temp\ipykernel_16992\397794853.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [9]:
trainer.train()

  0%|          | 0/52182 [00:00<?, ?it/s]

{'loss': 0.6925, 'grad_norm': 1.20574951171875, 'learning_rate': 4.999041815185313e-05, 'epoch': 0.0}
{'loss': 0.7103, 'grad_norm': 1.1004494428634644, 'learning_rate': 4.9980836303706265e-05, 'epoch': 0.0}
{'loss': 0.7083, 'grad_norm': 3.397369623184204, 'learning_rate': 4.997125445555939e-05, 'epoch': 0.0}
{'loss': 0.7001, 'grad_norm': 0.870637834072113, 'learning_rate': 4.996167260741252e-05, 'epoch': 0.0}
{'loss': 0.7123, 'grad_norm': 1.242234706878662, 'learning_rate': 4.9952090759265655e-05, 'epoch': 0.0}
{'loss': 0.7071, 'grad_norm': 0.9819422364234924, 'learning_rate': 4.994250891111878e-05, 'epoch': 0.0}
{'loss': 0.6984, 'grad_norm': 0.5825411677360535, 'learning_rate': 4.993292706297191e-05, 'epoch': 0.0}
{'loss': 0.7207, 'grad_norm': 0.9155024886131287, 'learning_rate': 4.992334521482504e-05, 'epoch': 0.0}
{'loss': 0.683, 'grad_norm': 2.076345205307007, 'learning_rate': 4.9913763366678166e-05, 'epoch': 0.01}
{'loss': 0.7078, 'grad_norm': 0.5070347785949707, 'learning_rate': 

  0%|          | 0/4349 [00:00<?, ?it/s]

{'eval_loss': 2.6886229515075684, 'eval_runtime': 104.4479, 'eval_samples_per_second': 666.131, 'eval_steps_per_second': 41.638, 'epoch': 1.0}
{'loss': 0.0371, 'grad_norm': 0.3170958161354065, 'learning_rate': 3.332758422444521e-05, 'epoch': 1.0}
{'loss': 0.1544, 'grad_norm': 10.541509628295898, 'learning_rate': 3.331800237629834e-05, 'epoch': 1.0}
{'loss': 0.058, 'grad_norm': 1.5102266073226929, 'learning_rate': 3.330842052815147e-05, 'epoch': 1.0}
{'loss': 0.1883, 'grad_norm': 0.14304222166538239, 'learning_rate': 3.32988386800046e-05, 'epoch': 1.0}
{'loss': 0.065, 'grad_norm': 12.017027854919434, 'learning_rate': 3.328925683185773e-05, 'epoch': 1.0}
{'loss': 0.0743, 'grad_norm': 0.046844396740198135, 'learning_rate': 3.327967498371086e-05, 'epoch': 1.0}
{'loss': 0.0801, 'grad_norm': 23.02384376525879, 'learning_rate': 3.327009313556399e-05, 'epoch': 1.0}
{'loss': 0.0449, 'grad_norm': 0.13162840902805328, 'learning_rate': 3.326051128741712e-05, 'epoch': 1.0}
{'loss': 0.1073, 'grad_no

  0%|          | 0/4349 [00:00<?, ?it/s]

{'eval_loss': 3.46482253074646, 'eval_runtime': 104.1889, 'eval_samples_per_second': 667.787, 'eval_steps_per_second': 41.741, 'epoch': 2.0}
{'loss': 0.0007, 'grad_norm': 0.018909599632024765, 'learning_rate': 1.6664750297037294e-05, 'epoch': 2.0}
{'loss': 0.0634, 'grad_norm': 0.03141848370432854, 'learning_rate': 1.6655168448890425e-05, 'epoch': 2.0}
{'loss': 0.0851, 'grad_norm': 0.013200709596276283, 'learning_rate': 1.6645586600743553e-05, 'epoch': 2.0}
{'loss': 0.0815, 'grad_norm': 0.37638530135154724, 'learning_rate': 1.663600475259668e-05, 'epoch': 2.0}
{'loss': 0.0008, 'grad_norm': 0.025775322690606117, 'learning_rate': 1.662642290444981e-05, 'epoch': 2.0}
{'loss': 0.0071, 'grad_norm': 0.38427653908729553, 'learning_rate': 1.661684105630294e-05, 'epoch': 2.0}
{'loss': 0.008, 'grad_norm': 0.8684228658676147, 'learning_rate': 1.660725920815607e-05, 'epoch': 2.0}
{'loss': 0.0391, 'grad_norm': 0.019330928102135658, 'learning_rate': 1.6597677360009202e-05, 'epoch': 2.0}
{'loss': 0.07

  0%|          | 0/4349 [00:00<?, ?it/s]

{'eval_loss': 3.9741897583007812, 'eval_runtime': 110.6034, 'eval_samples_per_second': 629.059, 'eval_steps_per_second': 39.321, 'epoch': 3.0}
{'train_runtime': 4745.3366, 'train_samples_per_second': 175.944, 'train_steps_per_second': 10.996, 'train_loss': 0.11443806514979535, 'epoch': 3.0}


TrainOutput(global_step=52182, training_loss=0.11443806514979535, metrics={'train_runtime': 4745.3366, 'train_samples_per_second': 175.944, 'train_steps_per_second': 10.996, 'total_flos': 5.491864436318208e+16, 'train_loss': 0.11443806514979535, 'epoch': 3.0})

In [10]:
results = trainer.evaluate()
print(results)

  0%|          | 0/4349 [00:00<?, ?it/s]

{'eval_loss': 3.9741897583007812, 'eval_runtime': 108.4132, 'eval_samples_per_second': 641.767, 'eval_steps_per_second': 40.115, 'epoch': 3.0}


In [11]:
model.save_pretrained("./fine_tuned_simcse_cites")
tokenizer.save_pretrained("./fine_tuned_simcse_cites")

('./fine_tuned_simcse_cites\\tokenizer_config.json',
 './fine_tuned_simcse_cites\\special_tokens_map.json',
 './fine_tuned_simcse_cites\\vocab.txt',
 './fine_tuned_simcse_cites\\added_tokens.json',
 './fine_tuned_simcse_cites\\tokenizer.json')

## Search for documents

In [21]:
# Load the saved model and tokenizer
model_path = "./fine_tuned_simcse_cites"
model = AutoModel.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [12]:
examples_dataframe = pd.read_excel(r"C:\Users\John\Desktop\Patent Datasets\final_data\patent_database_final_F03D.xlsx")

In [13]:
title_list = examples_dataframe["title"].tolist()
claims_list = examples_dataframe["claims"].tolist()
description_list = examples_dataframe["description"].tolist()
id_list = examples_dataframe["lens_id"].tolist()

In [14]:
examples_x = []
for i in range(len(title_list)):
    example = str(title_list[i]) + " " + str(claims_list[i]) + " " + str(description_list[i])
    examples_x.append(example)

print(examples_x[0])

BLADE MOUNTING CLAIMS: 1. An arrangement for in situ mounting of rotor blades to a rotor hub of a wind power plant comprising rotor blades, a rotor hub (40), a disc (1), a means for rotation (30) of the disc (1) around its axis, the disc (1) being in connected to rotor hub (40) in such a way that when the disc (1) rotates a certain degree of angle around its axis (3) it will cause the rotor hub (40) to rotate the same degree of angle around the axis of the rotor hub, the disc (1) being provided with a connecting device (2), characterized in that the disc (1) is caused to rotate by the means for rotation (30) and when the means for rotation (30) comes into a desired position, the disc (1) is secured by a securing means (20; 36, 37). 2. The arrangement according to claim 1, characterized in that the securing means comprises a male portion (23; 37) and a at least one female portion (2; 220; 36) respectively, and that one of the portions has a fixed position in relation to the disc (1) dur

In [15]:
chunk_size = 500
examples_x_new = []
id_list_new = []
title_list_new = []

for doc, doc_id, title in zip(examples_x, id_list, title_list):
    words = doc.split()  # Split document into words
    chunks = [words[i:i + chunk_size] for i in range(0, len(words), chunk_size)]  # Create chunks
    for chunk in chunks:
        examples_x_new.append(" ".join(chunk))  # Join words back into a string for each chunk
        id_list_new.append(doc_id)  # Append the same ID for each chunk
        title_list_new.append(title)  # Append the same title for each chunk

In [16]:
examples_x = examples_x_new
id_list= id_list_new
title_list = title_list_new

In [17]:
# Calculate the average number of words for all documents in examples_x
def average_word_count(documents):
    total_words = sum(len(doc.split()) for doc in documents)
    return total_words / len(documents)

# Assuming examples_x is already defined
average_words = average_word_count(examples_x)
print(f"The average number of words for all documents {len(examples_x)} in examples_x is {average_words:.2f}")

The average number of words for all documents 576369 in examples_x is 477.79


In [18]:
# Function to create SimCSE embeddings
def get_simcse_embeddings(text_list, batch_size=16):
    model.eval()  # Set model to evaluation mode
    embeddings = []

    with torch.no_grad():  # Disable gradient computation
        for i in range(0, len(text_list), batch_size):
            batch = text_list[i:i+batch_size]
            inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt")
            
            # Move tensors to GPU if available
            if torch.cuda.is_available():
                inputs = {key: val.cuda() for key, val in inputs.items()}
                model.cuda()
            
            # Get the embeddings (SimCSE uses the `pooler_output`)
            outputs = model(**inputs, return_dict=True)
            batch_embeddings = outputs.pooler_output.cpu()  # Detach from GPU
            
            embeddings.append(batch_embeddings)

    # Combine all embeddings into a single tensor
    return torch.cat(embeddings)

In [22]:
# Example usage
#title_list = ["This is a test sentence.", "Another sentence for testing.", "Final example sentence."]
simcse_embeddings = get_simcse_embeddings(examples_x[:2000], batch_size=16)

In [23]:
test_doc = examples_x[11601]

In [24]:
# Create embedding for the test title
test_doc_embedding = get_simcse_embeddings([test_doc])

# Compute cosine similarities
cosine_scores = cosine_similarity(test_doc_embedding, simcse_embeddings)[0]

# Get the top 10 similar titles
top_indices = cosine_scores.argsort()[-20:][::-1]

# Print the index, rank, score, lens_id, and title of each document
# print("Top 10 similar docs to '{}':".format(test_doc))
for rank, idx in enumerate(top_indices, start=1):
    score = cosine_scores[idx]
    lens_id = examples_dataframe.iloc[idx]['lens_id']
    title = examples_dataframe.iloc[idx]['title']
    print(f"Index: {idx}, Rank: {rank}, Score: {score:.4f}, Lens ID: {lens_id}, Title: {title}")

Index: 58, Rank: 1, Score: 0.9888, Lens ID: 006-589-585-423-044, Title: WIND POWER GENERATOR AND VENTILATION STRUCTURE OF THE SAME
Index: 427, Rank: 2, Score: 0.9884, Lens ID: 037-989-671-139-733, Title: METHOD FOR DETERMINING OPTIMUM VORTEX GENERATOR PLACEMENT FOR MAXIMUM EFFICIENCY ON A RETROFITTED WIND TURBINE GENERATOR OF UNKNOWN AERODYNAMIC DESIGN
Index: 1411, Rank: 3, Score: 0.9878, Lens ID: 121-988-708-268-804, Title: Apparatus and methods for handling rotor blades
Index: 1447, Rank: 4, Score: 0.9871, Lens ID: 129-330-100-716-832, Title: WIND TURBINE BLADE ASSEMBLY AND METHODS
Index: 428, Rank: 5, Score: 0.9863, Lens ID: 038-743-233-183-407, Title: Wind turbine blade monitoring systems
Index: 1056, Rank: 6, Score: 0.9863, Lens ID: 094-482-601-665-919, Title: Tower clearance monitoring system and method therefor
Index: 32, Rank: 7, Score: 0.9861, Lens ID: 003-015-344-477-947, Title: Wind assisted electric vehicle
Index: 505, Rank: 8, Score: 0.9860, Lens ID: 044-387-457-007-639, T

### FAISS ANN

In [25]:
# Convert to numpy for ANN or cosine similarity
simcse_embeddings_np = simcse_embeddings.numpy()

# Dimensionality of embeddings
embedding_dim = simcse_embeddings_np.shape[1]

# Convert the test title embedding to numpy
test_doc_embedding_np = test_doc_embedding.numpy()

In [26]:
# Create FAISS index for Euclidean distance
index = faiss.IndexFlatL2(embedding_dim)  # L2 for Euclidean distance
index.add(simcse_embeddings_np)

# Search for top 10 most similar titles using Euclidean distance
k = 10
distances, indices = index.search(test_doc_embedding_np, k)

# Retrieve and display similar titles
similar_titles = [id_list[idx] for idx in indices[0]]

for i, id in enumerate(similar_titles):
    print(f"Rank {i+1}: {id} (Distance: {distances[0][i]}) Title: {title_list[indices[0][i]]}")

Rank 1: 008-984-697-575-192 (Distance: 1.7491281032562256) Title: ELECTRIC MOTOR WINDING STRUCTURE, ELECTRIC MOTOR MAGNETIC CONDUCTIVE COMPONENT, FERROMAGNETIC LAMINATION AND ELECTRIC MOTOR
Rank 2: 000-729-551-090-695 (Distance: 1.860719919204712) Title: Wind turbine generator
Rank 3: 004-602-110-956-489 (Distance: 1.878331184387207) Title: Tunable apparatus for generating energy from a fluid flow induced movement of a surface structure relative to a frame with at least one adjustable frame portion
Rank 4: 001-182-027-086-817 (Distance: 1.9129066467285156) Title: MAIN DRIVE DEVICE OF PLANOCENTRIC SET FOR THREE ARMED FORCES
Rank 5: 003-703-771-552-55X (Distance: 1.9956899881362915) Title: A NACELLE FOR A WIND TURBINE
Rank 6: 003-703-771-552-55X (Distance: 2.1415445804595947) Title: A NACELLE FOR A WIND TURBINE
Rank 7: 011-833-275-435-181 (Distance: 2.1915249824523926) Title: Control of a multi-rotor wind turbine system using a central controller to calculate local control objectives
Ran